In [1]:
# Estevão Batista Sanchez RM 86649

In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn import svm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.read_csv("venda.csv")

In [4]:
df

,Unnamed: 0,X,date,mes,weekday,margem,venda,desconto,outdesc,outmg
0,1,1,20140801,agosto,sexta-feira,0.406111,110042.460,9190.907534,0,0
1,2,2,20140802,agosto,sabado,0.416022,58377.320,5713.043012,0,0
2,3,3,20140803,agosto,domingo,0.431993,64635.390,8621.708915,0,0
3,4,4,20140804,agosto,segunda-feira,0.409216,140417.321,18312.965640,0,0
4,5,5,20140805,agosto,terca-feira,0.449648,149700.286,19942.741300,0,0
...,...,...,...,...,...,...,...,...,...,...
391,392,392,20150827,agosto,quinta-feira,0.451562,NaN,5846.133551,0,0
392,393,393,20150828,agosto,sexta-feira,0.472608,NaN,2657.847261,0,0
393,394,394,20150829,agosto,sabado,0.385612,NaN,269.530000,0,0
394,395,395,20150830,agosto,domingo,0.495615,NaN,3434.838118,0,0


In [5]:
# Verificando valores NAN nas colunas
# Foi verificado que a coluna vendas possui 31 valores Nan

print('X',df.loc[pd.isna(df['X']),'X'].shape)
print('date',df.loc[pd.isna(df['date']),'date'].shape)
print('mes',df.loc[pd.isna(df['mes']),'mes'].shape)
print('weekday',df.loc[pd.isna(df['weekday']),'weekday'].shape)
print('margem',df.loc[pd.isna(df['margem']),'margem'].shape)
print('venda',df.loc[pd.isna(df['venda']),'venda'].shape)
print('desconto',df.loc[pd.isna(df['desconto']),'desconto'].shape)
print('outdesc',df.loc[pd.isna(df['outdesc']),'outdesc'].shape)
print('outmg',df.loc[pd.isna(df['outmg']),'outmg'].shape)


X (0,)
date (0,)
mes (0,)
weekday (0,)
margem (0,)
venda (31,)
desconto (0,)
outdesc (0,)
outmg (0,)


In [6]:
# Calculando o percentual de valores indefinidos 
# Os valores nan da coluna target(venda) representam 7,83% do total de linha do dataset
# Dessa forma, vamos optar por remover essas linhas

def calcula_percentual_indefinido(dataframe, coluna ):
    indefinidos = dataframe.loc[pd.isna(dataframe[coluna]), coluna].shape[0]
    total = dataframe[coluna].shape[0]

    return (indefinidos/total)* 100

In [7]:
calcula_percentual_indefinido(df, 'venda')

7.828282828282829

In [8]:
# Deletando as linhas com valor nan
df = df[np.isfinite(df['venda'])]

In [9]:
# Separando a coluna data, em ano, mes e dia
df['dia'] =  df['date'].apply(lambda x: str(x)[-2:])
df['ano'] =  df['date'].apply(lambda x: str(x)[:4])
df['mes'] =  df['date'].apply(lambda x: str(x)[4:6])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [10]:
# Verificando os valores unicos por coluna

print('mes',len(df.mes.unique()),df.mes.unique())
print('weekday',len(df.weekday.unique()),df.weekday.unique())
print('margem',len(df.margem.unique()),df.margem.unique())
print('venda',len(df.venda.unique()),df.venda.unique())
print('desconto',len(df.desconto.unique()),df.desconto.unique())
print('outdesc',len(df.outdesc.unique()),df.outdesc.unique())
print('outmg',len(df.outmg.unique()),df.outmg.unique())
print('dia',len(df.dia.unique()),df.dia.unique())
print('ano',len(df.ano.unique()),df.ano.unique())

mes 12 ['08' '09' '10' '11' '12' '01' '02' '03' '04' '05' '06' '07']
weekday 7 ['sexta-feira' 'sabado' 'domingo' 'segunda-feira' 'terca-feira'
 'quarta-feira' 'quinta-feira']
margem 365 [0.40611054 0.41602182 0.43199282 0.40921592 0.44964757 0.43939673
 0.42921606 0.45500243 0.44953982 0.4675417  0.42526276 0.42970555
 0.43740524 0.45647604 0.44147886 0.4017046  0.42096748 0.41811057
 0.43490101 0.3942519  0.41238224 0.41086882 0.41011619 0.42682028
 0.39419856 0.37945537 0.37764345 0.39979544 0.44599289 0.39644587
 0.42757707 0.37813313 0.4343589  0.40920811 0.40493769 0.41092017
 0.43478861 0.45436315 0.42479422 0.41989499 0.4372184  0.43627914
 0.42823603 0.41738201 0.44791524 0.44738072 0.42834922 0.4495104
 0.47722749 0.4517351  0.47012715 0.43119124 0.45946282 0.45908473
 0.47019789 0.42055777 0.43214112 0.42420296 0.4184709  0.4357133
 0.42173355 0.46765071 0.43489656 0.47268511 0.46779914 0.45638766
 0.43542504 0.45572491 0.45221968 0.47249759 0.45967017 0.50069217
 0.4553398  

In [11]:
# A partir da análise de valores unicos por coluna, determinamos quais colunas serão categorizadas e quais serão normalizadas

# Categorizando as colunas de mes, ano, dia, outdesc, outmg, weekday 

# Aplicando a função dummy nas colunas categóricas 

dummies_source = pd.get_dummies(df['outdesc'])
df['outdesc_1'] = dummies_source[0]
df['outdesc_2'] = dummies_source[1]

dummies_source = pd.get_dummies(df['outmg'])
df['outmg_1'] = dummies_source[0]
df['outmg_2'] = dummies_source[1]

dummies_source = pd.get_dummies(df['ano'])
df['ano_2014'] = dummies_source['2014']
df['ano_2015'] = dummies_source['2015']

dummies_source = pd.get_dummies(df['weekday'])
df['domingo'] = dummies_source['domingo']
df['segunda'] = dummies_source['segunda-feira']
df['terça'] = dummies_source['terca-feira']
df['quarta'] = dummies_source['quarta-feira']
df['quinta'] = dummies_source['quinta-feira']
df['sexta'] = dummies_source['sexta-feira']
df['sabado'] = dummies_source['sabado']


dummies_source = pd.get_dummies(df['dia'])
df['dia_01'] = dummies_source['01']
df['dia_02'] = dummies_source['02']
df['dia_03'] = dummies_source['03']
df['dia_04'] = dummies_source['04']
df['dia_05'] = dummies_source['05']
df['dia_06'] = dummies_source['06']
df['dia_07'] = dummies_source['07']
df['dia_08'] = dummies_source['08']
df['dia_09'] = dummies_source['09']
df['dia_10'] = dummies_source['10']
df['dia_11'] = dummies_source['11']
df['dia_12'] = dummies_source['12']
df['dia_13'] = dummies_source['13']
df['dia_14'] = dummies_source['14']
df['dia_15'] = dummies_source['15']
df['dia_16'] = dummies_source['16']
df['dia_17'] = dummies_source['17']
df['dia_18'] = dummies_source['18']
df['dia_19'] = dummies_source['19']
df['dia_20'] = dummies_source['20']
df['dia_21'] = dummies_source['21']
df['dia_22'] = dummies_source['22']
df['dia_23'] = dummies_source['23']
df['dia_24'] = dummies_source['24']
df['dia_25'] = dummies_source['25']
df['dia_26'] = dummies_source['26']
df['dia_27'] = dummies_source['27']
df['dia_28'] = dummies_source['28']
df['dia_29'] = dummies_source['29']
df['dia_30'] = dummies_source['30']
df['dia_31'] = dummies_source['31']


dummies_source = pd.get_dummies(df['mes'])
df['mes_01'] = dummies_source['01']
df['mes_02'] = dummies_source['02']
df['mes_03'] = dummies_source['03']
df['mes_04'] = dummies_source['04']
df['mes_05'] = dummies_source['05']
df['mes_06'] = dummies_source['06']
df['mes_07'] = dummies_source['07']
df['mes_08'] = dummies_source['08']
df['mes_09'] = dummies_source['09']
df['mes_10'] = dummies_source['10']
df['mes_11'] = dummies_source['11']
df['mes_12'] = dummies_source['12']











/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [12]:
# Normalizando as colunas margem, venda, desconto pela média

def normaliza_media(valores):
    media = np.mean(valores)
    minimo = np.min(valores)
    maximo = np.max(valores)
    return (valores - media)/(maximo-minimo)

In [13]:
df.margem = normaliza_media(df.margem)
df.venda = normaliza_media(df.venda)
df.desconto = normaliza_media(df.desconto)


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [14]:
df.head(1)

,Unnamed: 0,X,date,mes,weekday,margem,venda,desconto,outdesc,outmg,...,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09,mes_10,mes_11,mes_12
0,1,1,20140801,08,sexta-feira,-0.034669,0.052496,0.046766,0,0,...,0,0,0,0,0,1,0,0,0,0


In [15]:
# dropar as colunas que não serão utilizadas para o modelo

df.drop(columns=['X'],inplace=True)
df.drop(columns=['date'],inplace=True)
df.drop(columns=['ano'],inplace=True)
df.drop(columns=['mes'],inplace=True)
df.drop(columns=['weekday'],inplace=True)
df.drop(columns=['dia'],inplace=True)
df.drop(columns=['outdesc'],inplace=True)
df.drop(columns=['outmg'],inplace=True)
df.drop(columns=['Unnamed: 0'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
df.dtypes

margem       float64
venda        float64
desconto     float64
outdesc_1      uint8
outdesc_2      uint8
outmg_1        uint8
outmg_2        uint8
ano_2014       uint8
ano_2015       uint8
domingo        uint8
segunda        uint8
terça          uint8
quarta         uint8
quinta         uint8
sexta          uint8
sabado         uint8
dia_01         uint8
dia_02         uint8
dia_03         uint8
dia_04         uint8
dia_05         uint8
dia_06         uint8
dia_07         uint8
dia_08         uint8
dia_09         uint8
dia_10         uint8
dia_11         uint8
dia_12         uint8
dia_13         uint8
dia_14         uint8
dia_15         uint8
dia_16         uint8
dia_17         uint8
dia_18         uint8
dia_19         uint8
dia_20         uint8
dia_21         uint8
dia_22         uint8
dia_23         uint8
dia_24         uint8
dia_25         uint8
dia_26         uint8
dia_27         uint8
dia_28         uint8
dia_29         uint8
dia_30         uint8
dia_31         uint8
mes_01       

In [17]:
# Convertendo as colunas númericas que estão como tipo string em tipo int

df['outdesc_1'] = df['outdesc_1'].astype('int64')
df['outdesc_2'] = df['outdesc_2'].astype('int64')
df['outmg_1'] = df['outmg_1'].astype('int64')
df['outmg_2'] = df['outmg_2'].astype('int64')
df['ano_2014'] = df['ano_2014'].astype('int64')
df['ano_2015'] = df['ano_2015'].astype('int64')
df['domingo'] = df['domingo'].astype('int64')
df['segunda'] = df['segunda'].astype('int64')
df['terça'] = df['terça'].astype('int64')
df['quarta'] = df['quarta'].astype('int64')
df['quinta'] = df['quinta'].astype('int64')
df['sexta'] = df['sexta'].astype('int64')
df['sabado'] = df['sabado'].astype('int64')
df['dia_01'] = df['dia_01'].astype('int64')
df['dia_02'] = df['dia_02'].astype('int64')
df['dia_03'] = df['dia_03'].astype('int64')
df['dia_04'] = df['dia_04'].astype('int64')
df['dia_05'] = df['dia_05'].astype('int64')
df['dia_06'] = df['dia_06'].astype('int64')
df['dia_07'] = df['dia_07'].astype('int64')
df['dia_08'] = df['dia_08'].astype('int64')
df['dia_09'] = df['dia_09'].astype('int64')
df['dia_10'] = df['dia_10'].astype('int64')
df['dia_11'] = df['dia_11'].astype('int64')
df['dia_12'] = df['dia_12'].astype('int64')
df['dia_13'] = df['dia_13'].astype('int64')
df['dia_14'] = df['dia_14'].astype('int64')
df['dia_15'] = df['dia_15'].astype('int64')
df['dia_16'] = df['dia_16'].astype('int64')
df['dia_17'] = df['dia_17'].astype('int64')
df['dia_18'] = df['dia_18'].astype('int64')
df['dia_19'] = df['dia_19'].astype('int64')
df['dia_20'] = df['dia_20'].astype('int64')
df['dia_21'] = df['dia_21'].astype('int64')
df['dia_22'] = df['dia_22'].astype('int64')
df['dia_23'] = df['dia_23'].astype('int64')
df['dia_24'] = df['dia_24'].astype('int64')
df['dia_25'] = df['dia_25'].astype('int64')
df['dia_26'] = df['dia_26'].astype('int64')
df['dia_27'] = df['dia_27'].astype('int64')
df['dia_28'] = df['dia_28'].astype('int64')
df['dia_29'] = df['dia_29'].astype('int64')
df['dia_30'] = df['dia_30'].astype('int64')
df['dia_31'] = df['dia_31'].astype('int64')
df['mes_01'] = df['mes_01'].astype('int64')
df['mes_02'] = df['mes_02'].astype('int64')
df['mes_03'] = df['mes_03'].astype('int64')
df['mes_04'] = df['mes_04'].astype('int64')
df['mes_05'] = df['mes_05'].astype('int64')
df['mes_06'] = df['mes_06'].astype('int64')
df['mes_07'] = df['mes_07'].astype('int64')
df['mes_08'] = df['mes_08'].astype('int64')
df['mes_09'] = df['mes_09'].astype('int64')
df['mes_10'] = df['mes_10'].astype('int64')
df['mes_11'] = df['mes_11'].astype('int64')
df['mes_12'] = df['mes_12'].astype('int64')
df['mes_12'] = df['mes_12'].astype('int64')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [18]:
# verificando se todas as colunas estão com o tipo correto

df.dtypes

margem       float64
venda        float64
desconto     float64
outdesc_1      int64
outdesc_2      int64
outmg_1        int64
outmg_2        int64
ano_2014       int64
ano_2015       int64
domingo        int64
segunda        int64
terça          int64
quarta         int64
quinta         int64
sexta          int64
sabado         int64
dia_01         int64
dia_02         int64
dia_03         int64
dia_04         int64
dia_05         int64
dia_06         int64
dia_07         int64
dia_08         int64
dia_09         int64
dia_10         int64
dia_11         int64
dia_12         int64
dia_13         int64
dia_14         int64
dia_15         int64
dia_16         int64
dia_17         int64
dia_18         int64
dia_19         int64
dia_20         int64
dia_21         int64
dia_22         int64
dia_23         int64
dia_24         int64
dia_25         int64
dia_26         int64
dia_27         int64
dia_28         int64
dia_29         int64
dia_30         int64
dia_31         int64
mes_01       

In [19]:
# Verificando o tamanho do dataset para separar a porcetagem de treino e teste

print(len(df))

365


In [20]:
# Vamos utilizar 80% para treino e 20% para teste

qnt_treino = 0.8*365
qnt_teste = 0.2*365

print(qnt_treino)
print(qnt_teste)


292.0
73.0


In [21]:
# Separando o datafreme em treino e teste

treino = df.iloc[0:292]
teste = df.iloc[293:365]

In [22]:
df.columns

Index(['margem', 'venda', 'desconto', 'outdesc_1', 'outdesc_2', 'outmg_1',
       'outmg_2', 'ano_2014', 'ano_2015', 'domingo', 'segunda', 'terça',
       'quarta', 'quinta', 'sexta', 'sabado', 'dia_01', 'dia_02', 'dia_03',
       'dia_04', 'dia_05', 'dia_06', 'dia_07', 'dia_08', 'dia_09', 'dia_10',
       'dia_11', 'dia_12', 'dia_13', 'dia_14', 'dia_15', 'dia_16', 'dia_17',
       'dia_18', 'dia_19', 'dia_20', 'dia_21', 'dia_22', 'dia_23', 'dia_24',
       'dia_25', 'dia_26', 'dia_27', 'dia_28', 'dia_29', 'dia_30', 'dia_31',
       'mes_01', 'mes_02', 'mes_03', 'mes_04', 'mes_05', 'mes_06', 'mes_07',
       'mes_08', 'mes_09', 'mes_10', 'mes_11', 'mes_12'],
      dtype='object')

In [23]:
# Selecionando o target (coluna venda) e as colunas de análise para treino e teste. Selecionando as colunas para comparação dos modelos
y = treino.iloc[:,[1]]
x = treino.loc[:,['margem','desconto','outdesc_1','outdesc_2','outmg_1','outmg_2','domingo','segunda','terça','quarta','quinta','sexta','sabado']]
X_prev = teste.loc[:,['margem','desconto','outdesc_1','outdesc_2','outmg_1','outmg_2','domingo','segunda','terça','quarta','quinta','sexta','sabado']]

x_2 = treino.loc[:,['margem','desconto','outdesc_1','outdesc_2','outmg_1','outmg_2','domingo','segunda','terça','quarta','quinta','sexta','sabado','mes_01','mes_02','mes_03','mes_04','mes_05','mes_06','mes_07','mes_08','mes_09','mes_10','mes_11','mes_12']]
X_prev_2 = teste.loc[:,['margem','desconto','outdesc_1','outdesc_2','outmg_1','outmg_2','domingo','segunda','terça','quarta','quinta','sexta','sabado','mes_01','mes_02','mes_03','mes_04','mes_05','mes_06','mes_07','mes_08','mes_09','mes_10','mes_11','mes_12']]

x_3 = treino.loc[:,['margem','desconto','outdesc_1','outdesc_2','outmg_1','outmg_2']]
X_prev_3 = teste.loc[:,['margem','desconto','outdesc_1','outdesc_2','outmg_1','outmg_2']]

x_4 = treino.loc[:,['margem','desconto','outdesc_1','outdesc_2']]
X_prev_4 = teste.loc[:,['margem','desconto','outdesc_1','outdesc_2']]

In [24]:
# Cálculo das estatísticas da regressão linear para o primeiro modelo

mod = sm.OLS(y, x).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  venda   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     103.7
Date:                Mon, 27 Jun 2022   Prob (F-statistic):           3.42e-88
Time:                        19:37:49   Log-Likelihood:                 555.04
No. Observations:                 292   AIC:                            -1088.
Df Residuals:                     281   BIC:                            -1048.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
margem        -0.1500      0.015     -9.931      0.0

In [25]:
# Cálculo das estatísticas da regressão linear para o segundo modelo

mod_2 = sm.OLS(y, x_2).fit()
print(mod_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  venda   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.829
Method:                 Least Squares   F-statistic:                     75.26
Date:                Mon, 27 Jun 2022   Prob (F-statistic):           1.75e-96
Time:                        19:37:56   Log-Likelihood:                 597.14
No. Observations:                 292   AIC:                            -1154.
Df Residuals:                     272   BIC:                            -1081.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
margem        -0.1895      0.019     -9.731      0.0

In [26]:
# Cálculo das estatísticas da regressão linear para o terceiro modelo

mod_3 = sm.OLS(y, x_3).fit()
print(mod_3.summary())

                            OLS Regression Results                            
Dep. Variable:                  venda   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     202.8
Date:                Mon, 27 Jun 2022   Prob (F-statistic):           2.44e-82
Time:                        19:37:58   Log-Likelihood:                 525.37
No. Observations:                 292   AIC:                            -1041.
Df Residuals:                     287   BIC:                            -1022.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
margem        -0.1692      0.016    -10.390      0.0

In [27]:
# Cálculo das estatísticas da regressão linear para o quarto modelo

mod_4 = sm.OLS(y, x_4).fit()
print(mod_4.summary())


                            OLS Regression Results                            
Dep. Variable:                  venda   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     256.9
Date:                Mon, 27 Jun 2022   Prob (F-statistic):           4.37e-81
Time:                        19:37:59   Log-Likelihood:                 519.51
No. Observations:                 292   AIC:                            -1031.
Df Residuals:                     288   BIC:                            -1016.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
margem        -0.1557      0.016     -9.674      0.0

In [28]:
# Construção do modelo de regressão Linear para o primeiro modelo

mod1 = LinearRegression()
mod1.fit(x,y)

LinearRegression()

In [29]:
# Construção do modelo de regressão Linear para o segundo modelo

mod2 = LinearRegression()
mod2.fit(x_2,y)

LinearRegression()

In [30]:
# Construção do modelo de regressão Linear para o terceiro modelo

mod3 = LinearRegression()
mod3.fit(x_3,y)

LinearRegression()

In [31]:
# Construção do modelo de regressão Linear para o quarto modelo

mod4 = LinearRegression()
mod4.fit(x_4,y)


LinearRegression()

In [32]:
# Preparação da base de teste para previsão utilizando o modelo de regressão para o primeiro modelo
Y_prev = mod1.predict(X_prev)
Y_real = teste.iloc[:,[1]].to_numpy()

In [33]:
# Preparação da base de teste para previsão utilizando o modelo de regressão para o segundo modelo

Y_prev_2 = mod2.predict(X_prev_2)
Y_real = teste.iloc[:,[1]].to_numpy()

In [34]:
# Preparação da base de teste para previsão utilizando o modelo de regressão para o terceiro modelo

Y_prev_3 = mod3.predict(X_prev_3)
Y_real = teste.iloc[:,[1]].to_numpy()

In [35]:
# Preparação da base de teste para previsão utilizando o modelo de regressão para o quarto modelo
Y_prev_4 = mod4.predict(X_prev_4)
Y_real = teste.iloc[:,[1]].to_numpy()

In [36]:
# Resultados primeiro modelo

# Calculo do erro de previsão
erro = Y_real - Y_prev
# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro))
print(erro_abs_medio)
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_real)
print(erro_medio_perc)

0.022495599692867085
-0.42770118520792244


In [37]:
#Resultados segundo modelo

# Calculo do erro de previsão
erro_2 = Y_real - Y_prev_2
# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro_2))
print(erro_abs_medio)
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_real)
print(erro_medio_perc)



4898260184660.971
-93128954775122.33


In [38]:
#Resultados terceiro modelo

# Calculo do erro de previsão
erro_3 = Y_real - Y_prev_3
# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro_3))
print(erro_abs_medio)
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_real)
print(erro_medio_perc)

0.02463538531087507
-0.46838420132698466


In [39]:
#Resultados quarto modelo

# Calculo do erro de previsão
erro_4 = Y_real - Y_prev_4
# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro_4))
print(erro_abs_medio)
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_real)
print(erro_medio_perc)


0.024014230843439285
-0.45657440271984995
